In [53]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import string
from nltk.corpus import *
from wordcloud import *
from sklearn import *
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import *
from sklearn.tree import *
from sklearn.feature_extraction.text import *
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import *
from sklearn.metrics import *
%matplotlib inline

In [54]:
df=pd.read_csv('one_hot.csv')
import matplotlib.pyplot as plt
df.head()
len(df)

2026

In [55]:
df.corr()['posttest']#the correlation with the target variable

n_student                                -0.505762
pretest                                   0.951527
posttest                                  1.000000
school_ANKYI                              0.043920
school_CCAAW                              0.180962
                                            ...   
teaching_method_Standard                 -0.312110
gender_Female                            -0.002834
gender_Male                               0.002834
lunch_Does not qualify                    0.602951
lunch_Qualifies for reduced/free lunch   -0.602951
Name: posttest, Length: 134, dtype: float64

In [56]:
cor = df.corr()
#Correlation with output variable
cor_target = abs(cor["posttest"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.3]#choosing the
relevant_features

n_student                                 0.505762
pretest                                   0.951527
posttest                                  1.000000
school_GOOBU                              0.358480
school_IDGFP                              0.307869
school_KZKKE                              0.317906
school_UKPGS                              0.438616
school_VVTVA                              0.306937
school_setting_Suburban                   0.455322
school_setting_Urban                      0.334434
school_type_Non-public                    0.375162
school_type_Public                        0.375162
teaching_method_Experimental              0.312110
teaching_method_Standard                  0.312110
lunch_Does not qualify                    0.602951
lunch_Qualifies for reduced/free lunch    0.602951
Name: posttest, dtype: float64

In [57]:
best_features=["n_student",'pretest','school_GOOBU','school_IDGFP','school_KZKKE','school_UKPGS','school_VVTVA',
               'school_setting_Suburban','school_setting_Urban','school_type_Non-public','school_type_Public',
               'teaching_method_Experimental','teaching_method_Standard','lunch_Does not qualify',
               'lunch_Qualifies for reduced/free lunch']

In [58]:
X=df[best_features]
Y=df['posttest']
X_train, X_test, Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=65,shuffle=True)

In [59]:

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
model = LinearRegression()
model.fit(X_train,Y_train)
pred = model.predict(X_test)
predicted_values = np.array(pred)
# Iterate through the list and round each value to the nearest integer
rounded_values = np.round(predicted_values).astype(int)

# Calculate the mean squared error (MSE)
mse = mean_squared_error(Y_test, rounded_values)

# Calculate the root mean squared error (RMSE)
rmse = np.sqrt(mse)

# Calculate the explained variance score (R^2)
r2 = r2_score(Y_test, rounded_values)

# Calculate the fraction of variance explained (FVE)
fve = 1 - mse / np.var(Y_test)

# Print the evaluation metrics
print('MSE: ', mse)
print('RMSE: ', rmse)
print('R^2: ', r2)
print(f'Accuracy: {round(r2_score(Y_test , rounded_values),3)*100}%')

print('FVE: ', fve)






MSE:  10.238916256157635
RMSE:  3.1998306605440288
R^2:  0.9489727765335639
Accuracy: 94.89999999999999%
FVE:  0.9489727765335639


In [60]:
#now for random forest regressor
rfr=GridSearchCV(RandomForestRegressor(random_state=62),{
    # Number of trees in random forest
    'n_estimators': [100, 200, 300],
    'criterion': ['squared_error', 'poisson','absolute_error'],
    # Maximum depth of tree
    'max_depth': [10, 20, 30, 50],
    # minimum number of samples needed to split
    'min_samples_split': [2, 5, 10],
    # Number of features to consider at every split
    'max_features' : ['log2', 'sqrt',1],
    # Method of selecting samples for training each tree
    'bootstrap' : [True, False],
},cv=3,return_train_score=False)
rfr.fit(X_train,Y_train)




GridSearchCV(cv=3, estimator=RandomForestRegressor(random_state=62),
             param_grid={'bootstrap': [True, False],
                         'criterion': ['squared_error', 'poisson',
                                       'absolute_error'],
                         'max_depth': [10, 20, 30, 50],
                         'max_features': ['log2', 'sqrt', 1],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]})

In [61]:
rfr.best_params_

{'bootstrap': True,
 'criterion': 'squared_error',
 'max_depth': 10,
 'max_features': 'log2',
 'min_samples_split': 10,
 'n_estimators': 300}

In [24]:
from sklearn.ensemble import RandomForestRegressor
from math import log2
rfr_best=RandomForestRegressor(criterion="squared_error",max_depth=10,n_estimators=300, random_state=62,bootstrap=True,
                           max_features='log2',min_samples_split=10)
rfr_best.fit(X_train,Y_train)
pred1 = rfr_best.predict(X_test)
predicted_values1 = np.array(pred1)
# Iterate through the list and round each value to the nearest integer
rounded_values1 = np.round(predicted_values1).astype(int)
# Calculate the mean squared error (MSE)
mse = mean_squared_error(Y_test, rounded_values1)

# Calculate the root mean squared error (RMSE)
rmse = np.sqrt(mse)

# Calculate the explained variance score (R^2)
r2 = r2_score(Y_test, rounded_values1)

# Calculate the fraction of variance explained (FVE)
fve = 1 - mse / np.var(Y_test)

# Print the evaluation metrics
print('MSE: ', mse)
print('RMSE: ', rmse)
print('R^2: ', r2)
print(f'Accuracy: {round(r2_score(Y_test , rounded_values1),3)*100}%')

print('FVE: ', fve)

MSE:  9.002463054187192
RMSE:  3.000410480948764
R^2:  0.9551348323863786
Accuracy: 95.5%
FVE:  0.9551348323863786


In [40]:
#now for decision Tree regression
from sklearn.tree import DecisionTreeRegressor
dtr=GridSearchCV(DecisionTreeRegressor(random_state=62),{
    'criterion': ['squared_error', 'poisson','absolute_error'],
    # Maximum depth of tree
    'max_depth': [10, 20, 30, 50],
    # minimum number of samples needed to split
    'min_samples_split': [2, 5, 10],
    # Number of features to consider at every split
    'max_features' : ['log2', 'sqrt',1]
},cv=3,return_train_score=False)
dtr.fit(X_train,Y_train)




GridSearchCV(cv=3, estimator=DecisionTreeRegressor(random_state=62),
             param_grid={'criterion': ['squared_error', 'poisson',
                                       'absolute_error'],
                         'max_depth': [10, 20, 30, 50],
                         'max_features': ['log2', 'sqrt', 1],
                         'min_samples_split': [2, 5, 10]})

In [39]:
dtr.best_params_

{'criterion': 'squared_error',
 'max_depth': 10,
 'max_features': 'log2',
 'min_samples_split': 5}

In [51]:
from math import log2
dtr_best=DecisionTreeRegressor(criterion="squared_error",max_depth=10, random_state=62,
                           max_features='log2',min_samples_split=5)
dtr_best.fit(X_train,Y_train)
pred2 = dtr_best.predict(X_test)
predicted_values2 = np.array(pred2)
# Iterate through the list and round each value to the nearest integer
rounded_values2 = np.round(predicted_values2).astype(int)
# Calculate the mean squared error (MSE)
mse = mean_squared_error(Y_test, rounded_values2)

# Calculate the root mean squared error (RMSE)
rmse = np.sqrt(mse)

# Calculate the explained variance score (R^2)
r2 = r2_score(Y_test, rounded_values2)

# Calculate the fraction of variance explained (FVE)
fve = 1 - mse / np.var(Y_test)

# Print the evaluation metrics
print('MSE: ', mse)
print('RMSE: ', rmse)
print('R^2: ', r2)
print(f'Accuracy: {round(r2_score(Y_test , rounded_values2),3)*100}%')

print('FVE: ', fve)

MSE:  12.509852216748769
RMSE:  3.536926945350832
R^2:  0.9376552157839719
Accuracy: 93.8%
FVE:  0.9376552157839719


In [64]:
#now for ada boost regression
from sklearn.ensemble import AdaBoostRegressor
ada=AdaBoostRegressor()
adbr=GridSearchCV(AdaBoostRegressor(random_state=62),{
    
    # Number of trees in random forest
    'n_estimators': [100, 200, 300],
    #Weight applied to each regressor at each boosting iteration
    'learning_rate':[.001,0.01,.1],
    #The loss function to use when updating the weights after each boosting iteration.
    'loss': ['linear', 'square', 'exponential'],
    
    },cv=3,return_train_score=False)
adbr.fit(X_train,Y_train)

GridSearchCV(cv=3, estimator=AdaBoostRegressor(random_state=62),
             param_grid={'learning_rate': [0.001, 0.01, 0.1],
                         'loss': ['linear', 'square', 'exponential'],
                         'n_estimators': [100, 200, 300]})

In [65]:
adbr.best_params_

{'learning_rate': 0.1, 'loss': 'square', 'n_estimators': 200}

In [66]:
from math import log2
adbr_best=AdaBoostRegressor(loss='square', random_state=62,n_estimators=200,
                           learning_rate= 0.1)
adbr_best.fit(X_train,Y_train)
pred3 = adbr_best.predict(X_test)
predicted_values3 = np.array(pred3)
# Iterate through the list and round each value to the nearest integer
rounded_values3 = np.round(predicted_values3).astype(int)
# Calculate the mean squared error (MSE)
mse = mean_squared_error(Y_test, rounded_values3)

# Calculate the root mean squared error (RMSE)
rmse = np.sqrt(mse)

# Calculate the explained variance score (R^2)
r2 = r2_score(Y_test, rounded_values3)

# Calculate the fraction of variance explained (FVE)
fve = 1 - mse / np.var(Y_test)

# Print the evaluation metrics
print('MSE: ', mse)
print('RMSE: ', rmse)
print('R^2: ', r2)
print(f'Accuracy: {round(r2_score(Y_test , rounded_values3),3)*100}%')

print('FVE: ', fve)

MSE:  12.748768472906404
RMSE:  3.570541761820803
R^2:  0.9364645396530495
Accuracy: 93.60000000000001%
FVE:  0.9364645396530495
